In [ ]:
from transformers import GPT2TokenizerFast
new_tokenizer = GPT2TokenizerFast.from_pretrained("openai-community/gpt2", cache_dir="hfcache")

In [ ]:
from model import YALMGPT2Model, YALMGPT2Config
config = YALMGPT2Config(n_positions=1024,n_layer=12,n_embd=768,n_head=8)
model = YALMGPT2Model(config)

In [ ]:
from datasets import load_dataset
ds = load_dataset("facebook/belebele", "zho_Hans", cache_dir="hfcache", split="test")

In [ ]:
ds2=ds.rename_column('flores_passage', 'text')

In [ ]:
new_tokenizer.model_max_length = config.n_positions
new_tokenizer.pad_token = new_tokenizer.eos_token

from coach import Coach

def tokenize_function(example):
    return new_tokenizer(example["text"], truncation=False)

tokenized_datasets = ds2.map(tokenize_function, batched=True)

coach = Coach(tokenized_datasets, new_tokenizer)

In [ ]:
modified_dataset = tokenized_datasets.map(coach)

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./t2",
    label_names=["output_ids","dni_labels"]
)
trainer = Trainer(
    model=model,
    train_dataset=modified_dataset,
    args=training_args,
)

In [ ]:
trainer.train()